In [1]:
import pandas as pd
import numpy as np
import warnings

pd.options.mode.chained_assignment = None

warnings.simplefilter(action='ignore', category=FutureWarning)

item_master_excel_file = '../data/inzi/Item Master_08_18_2021 0317.xls'
material_history_excel_file = '../data/inzi/Material Movement History_08_19_2021 1026.xls'
out_put_file = '../output/9-15.xlsx'

item_master = pd.read_excel(
    item_master_excel_file, index_col=False, header=1
)
material_history = pd.read_excel(
    material_history_excel_file, index_col=False, header=1
)

# Item master excel file must have 3 columns 'Material', 'Material Type', 'Procurement'
temp_item_master = item_master[['Material', 'Material Type', 'Procurement']]

'''
PREPARE FOR STEP ONE
'''
step_one = pd.merge(material_history, temp_item_master,
                    on="Material", how="left")

step_one.rename(columns={
    'Reference': 'Order Category',
    'Unnamed: 13': 'Order Data',
    'Unnamed: 14': 'Master Category',
    'Unnamed: 15': 'Master Data',
    'Unnamed: 16': 'Remark'
}, inplace=True)

step_one = step_one.iloc[1:, :]

# pd.set_option('mode.chained_assignment', None)

step_one.loc[step_one['Quantity'] > 0, 'Input'] = step_one['Quantity']
step_one.loc[step_one['Quantity'] > 0, 'Output'] = 0
step_one.loc[step_one['Quantity'] <= 0, 'Input'] = 0
step_one.loc[step_one['Quantity'] <= 0, 'Output'] = -step_one['Quantity']

cols = step_one.columns.tolist()

cols = cols[:10] + [cols[-2], cols[-1]] + cols[10:-2]

step_one = step_one[cols]

step_one['Account code'] = [int(x)
                            for x in (step_one['Movement Type'].str[:3])]

cols = step_one.columns.tolist()
cols = cols[:7] + [cols[-1]] + cols[7: -1]
step_one = step_one[cols]

columns_name = {
    'H': 'Account code',
    'p': 'Order Category',
    'W': 'Material Type',
    'X': 'Procurement'
}



In [2]:
step_one

,Unnamed: 0,Plant,Entry Date,Material,Location,Stock Type,Movement Type,Account code,Posting Date,Unit,...,Provide,Order Category,Order Data,Master Category,Master Data,Remark,Previous Stock,Entry User,Material Type,Procurement
1,1.0,DYKV,08/18/2021 20:01:17,BN96-52592A,RSUB,Unrestricted,101 (GR for Order),101,08/18/2021,PC,...,,Purchase Order,3000023377-70,Vendor,P551,,21394.0,RSUB_MANH,ROH,F
2,2.0,DYKV,08/18/2021 20:01:06,BN96-52591A,RSUB,Unrestricted,101 (GR for Order),101,08/18/2021,PC,...,,Purchase Order,3000023377-60,Vendor,P551,,32904.0,RSUB_MANH,ROH,F
3,3.0,DYKV,08/18/2021 20:00:42,BN96-52584A,RSUB,Unrestricted,101 (GR for Order),101,08/18/2021,PC,...,,Purchase Order,3000023377-40,Vendor,P551,,65776.0,RSUB_MANH,ROH,F
4,4.0,DYKV,08/18/2021 20:00:33,BN96-52583A,RSUB,Unrestricted,101 (GR for Order),101,08/18/2021,PC,...,,Purchase Order,3000023377-30,Vendor,P551,,35924.0,RSUB_MANH,ROH,F
5,5.0,DYKV,08/18/2021 20:00:20,BN96-46034A,RSUB,Unrestricted,101 (GR for Order),101,08/18/2021,PC,...,,Purchase Order,3000023377-20,Vendor,P551,,39532.0,RSUB_MANH,ROH,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4677,4677.0,DYKV,08/18/2021 06:58:07,BN96-50479B,FPRE,Unrestricted,601 (GI for Outbound Delivery),601,08/18/2021,PC,...,,Sales Order,4000029928-30,Customer,P551,,521.0,cuongwh,FERT,E
4678,4678.0,DYKV,08/18/2021 05:41:05,BN44-01154A,WPAS,Unrestricted,401 (Stock transfer SLoc to SLoc),401,08/18/2021,PC,...,,WC,ASSYPR-17,Cost Center,ASSYPR-17,"08/14/2021~08/20/2021,~,,54012",1.0,RSUB_MANH,ROH,F
4679,4679.0,DYKV,08/18/2021 05:41:05,BN44-01154A,RSUB,Unrestricted,401 (Stock transfer SLoc to SLoc),401,08/18/2021,PC,...,,WC,ASSYPR-17,Cost Center,ASSYPR-17,"08/14/2021~08/20/2021,~,,54012",102.0,RSUB_MANH,ROH,F
4680,4680.0,DYKV,08/18/2021 05:40:20,BN96-46053A,WPAS,Unrestricted,401 (Stock transfer SLoc to SLoc),401,08/18/2021,PC,...,,,,,,GI PKL 7443 THỰC TẾ PHÁT 1624EA,680.0,RSUB_MANH,ROH,F


In [3]:
aggregation_functions = {}
columns_order = [101, 102, 103, 321, 323, 327, 343, 344, 401, 602, 701,
                     720, 801, 809, 201, 261, 555, 601, 609, 702, 712, 721, 803]

step_one.fillna(0, inplace=True)

for column in pd.unique(step_one['Account code']):
    if isinstance(column, np.int64):
        print(column, type(column))
        aggregation_functions[column] = 'sum'
        if column in [321, 323, 327, 343, 344, 401]:
            aggregation_functions[column] = 'first'



101 <class 'numpy.int64'>
261 <class 'numpy.int64'>
401 <class 'numpy.int64'>
343 <class 'numpy.int64'>
609 <class 'numpy.int64'>
601 <class 'numpy.int64'>
720 <class 'numpy.int64'>
344 <class 'numpy.int64'>
721 <class 'numpy.int64'>
201 <class 'numpy.int64'>
623 <class 'numpy.int64'>
321 <class 'numpy.int64'>


In [4]:
aggregation_functions

{101: 'sum',
 261: 'sum',
 401: 'first',
 343: 'first',
 609: 'sum',
 601: 'sum',
 720: 'sum',
 344: 'first',
 721: 'sum',
 201: 'sum',
 623: 'sum',
 321: 'first'}

In [5]:
summary = step_one.pivot_table(index=["Material"],
                            columns="Account code", values="Quantity", aggfunc=aggregation_functions)


summary.fillna(0, inplace=True)
in_columns = []
out_columns = []


for column in summary.columns:
    if column in [101, 602, 720, 801, 809]:
        in_columns.append(column)
    if column in [102, 201, 261, 555, 601, 609, 721, 803]:
        out_columns.append(column)

summary['IN'] = summary[in_columns].sum(axis=1)
summary['OUT'] = summary[out_columns].sum(axis=1)

KeyError: 'Column(s) [101, 201, 261, 321, 343, 344, 401, 601, 609, 623, 720, 721] do not exist'

In [ ]:
summary